In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_community.embeddings import HuggingFaceEmbeddings as CommunityHuggingFaceEmbeddings

In [2]:
load_dotenv()
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "travel-assistant-index"
index = pc.Index(index_name)
embeddings = CommunityHuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\Amanda\AppData\Local\Temp\ipykernel_15368\2831825917.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = CommunityHuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [3]:
data_dir = "./data"
documents = []
for filename in os.listdir(data_dir):
    if filename.endswith(".txt"):
        file_path = os.path.join(data_dir, filename)
        loader = TextLoader(file_path, encoding="utf-8")
        documents.extend(loader.load())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)
print(f"Divididos {len(documents)} documentos em {len(docs)} chunks.")

vectors = []
for i, doc in enumerate(docs):
    embedding = embeddings.embed_query(doc.page_content)
    vector = {
        "id": f"doc-{i}",
        "values": embedding,
        "metadata": {"source": doc.metadata.get("source"), "text": doc.page_content}
    }
    vectors.append(vector)

batch_size = 100
for i in range(0, len(vectors), batch_size):
    batch = vectors[i:i+batch_size]
    index.upsert(vectors=batch)
    print(f"Lote {i//batch_size + 1} enviado.")

Divididos 5 documentos em 10 chunks.
Lote 1 enviado.
